# Data Cleaning for CFPS data year 2010-2016

## Datasets used

From Dropbox: China-Education-Wages -> Data -> CFPS Data 2010-2016

Downloaded:

- 2010 Egnlish -> ecfps2010adult_112014.dta
  - Renamed 2010adult.dta
- 2012 Egnlish -> ecfps2012adultcombined_032015.dta
  - Renamed 2012adult.dta
- 2014 English -> ecfps2014adult_170630.dta
  - Renamed 2014adult.dta
- 2016 English -> ecfps2016adult_201808.dta
  - Renamed 2016adult.dta


In [7]:
library(knitr)
library(xtable)
library(broom)
library(dplyr)
library(tidyverse)
library(ggplot2)
library(stargazer)
library(lubridate)
library(haven)
library(ineq)
library(PerformanceAnalytics)
library(gglorenz)

# Reading in the datasets and renaming without filtering anything


In [31]:
data10 <- read_dta("~/Downloads/ecfps2010adult_112014.dta")
data10 <- filter(data10, qc1 > 0 & income > 0)
data12 <- read_dta("~/Downloads/ecfps2012adultcombined_032015.dta")
data12 <- filter(data12, income > 0)
data14 <- read_dta("~/Downloads/ecfps2014adult_170630.dta")
data14 <- filter(data14, p_income > 0)
data16 <- read_dta("~/Downloads/ecfps2016adult_201808.dta")
data16 <- filter(data16, incomea > 0 | income > 0)

In [32]:
edu_2010 <- data.frame("pid" = data10$pid, "edu10" = data10$qc1)
edu_2012 <- data.frame("pid" = data12$pid, "edu12" = data12$sch2012)
edu_2014 <- data.frame("pid" = data14$pid, "edu14" = data14$pw1r)
edu_2016 <- data.frame("pid" = data16$pid, "edu16" = data16$cfps2016edu)

latest_edu <- merge(merge(merge(edu_2010, edu_2012, all = TRUE),
 edu_2014, all = TRUE), edu_2016, all = TRUE)

In [33]:
for (row in seq_len(nrow(latest_edu))) {
  edu10 <- latest_edu[row, "edu10"]
  edu12 <- latest_edu[row, "edu12"]
  if (is.na(edu12) && !is.na(edu10)) {
    latest_edu[row, "edu12"] <- edu10
    edu12 <- edu10
  }
  edu14 <- latest_edu[row, "edu14"]
  if (is.na(edu14) && !is.na(edu12)) {
    latest_edu[row, "edu14"] <- edu12
    edu14 <- edu12
  }
  edu16 <- latest_edu[row, "edu16"]
  if (is.na(edu16) && !is.na(edu14)) {
    latest_edu[row, "edu16"] <- edu14
  }
}

In [34]:
mincer10 <- data.frame(
  pid = data10$pid,
  income = data10$income,
  age = data10$qa1age,
  gender = data10$gender,
  urban = data10$urban,
  prov = data10$provcd,
  ethnic = data10$qa5code,
  married = 0,
  party = 0,
  edu = data10$qc1,
  postsecondary = 0,
  seniorsecondary = 0,
  juniorsecondary = 0,
  primary = 0,
  illiterate = 0,
  y10 = 1,
  y12 = 0,
  y14 = 0,
  y16 = 0,
  Shanghai = 0,
  Liaoning = 0,
  Guangdong = 0,
  Gansu = 0,
  Others = 0)

for (row in seq_len(nrow(data10))) {
  marriage <- data10[row, "qe1"]
  if (marriage == 2) {
    mincer10[row, "married"] <- 1
  }
  party <- data10[row, "qa7_s_1"]
  if (party == 1) {
    mincer10[row, "party"] <- 1
  }
  edu <- mincer10[row, "edu"]
  if (edu == 1) {
    mincer10[row, "illiterate"] <- 1
  } else if (edu == 2) {
    mincer10[row, "primary"] <- 1
  } else if (edu == 3) {
    mincer10[row, "juniorsecondary"] <- 1
  } else if (edu == 4) {
    mincer10[row, "seniorsecondary"] <- 1
  } else if (edu > 4) {
    mincer10[row, "postsecondary"] <- 1
  }
  prov <- mincer10[row, "prov"]
  if (prov == 21) {
    mincer10[row, "Liaoning"] <- 1
  } else if (prov == 31) {
    mincer10[row, "Shanghai"] <- 1
  } else if (prov == 44) {
    mincer10[row, "Guangdong"] <- 1
  } else if (prov == 62) {
    mincer10[row, "Gansu"] <- 1
  } else {
    mincer10[row, "Others"] <- 1
  }
}

In [35]:
mincer12 <- data.frame(
  pid = data12$pid,
  income = data12$income,
  age = data12$cfps2012_age,
  gender = data12$cfps2012_gender,
  urban = data12$urban12,
  prov = data12$provcd,
  ethnic = data12$qa701code,
  married = 0,
  party = data12$sn401,
  postsecondary = 0,
  seniorsecondary = 0,
  juniorsecondary = 0,
  primary = 0,
  illiterate = 0,
  y10 = 0,
  y12 = 1,
  y14 = 0,
  y16 = 0,
  Shanghai = 0,
  Liaoning = 0,
  Guangdong = 0,
  Gansu = 0,
  Others = 0)

mincer12 <- merge(mincer12, latest_edu, all.x = TRUE)
mincer12 <- filter(mincer12, edu12 > 0)
mincer12$edu <- mincer12$edu12
mincer12 <- select(mincer12, -edu10, -edu12, -edu14, -edu16)


for (row in seq_len(nrow(mincer12))) {
  marriage <- data12[row, "qe104"]
  if (marriage == 2) {
    mincer12[row, "married"] <- 1
  }
  edu <- mincer12[row, "edu"]
  if (edu == 1) {
    mincer12[row, "illiterate"] <- 1
  } else if (edu == 2) {
    mincer12[row, "primary"] <- 1
  } else if (edu == 3) {
    mincer12[row, "juniorsecondary"] <- 1
  } else if (edu == 4) {
    mincer12[row, "seniorsecondary"] <- 1
  } else if (edu > 4) {
    mincer12[row, "postsecondary"] <- 1
  }
  prov <- mincer12[row, "prov"]
  if (prov == 21) {
    mincer12[row, "Liaoning"] <- 1
  } else if (prov == 31) {
    mincer12[row, "Shanghai"] <- 1
  } else if (prov == 44) {
    mincer12[row, "Guangdong"] <- 1
  } else if (prov == 62) {
    mincer12[row, "Gansu"] <- 1
  } else {
    mincer12[row, "Others"] <- 1
  }
}

In [36]:
mincer14 <- data.frame(
  pid = data14$pid,
  income = data14$p_income,
  age = data14$cfps2014_age,
  gender = data14$cfps_gender,
  urban = data14$urban14,
  prov = data14$provcd14,
  ethnic = data14$cfps_minzu,
  married = 0,
  party = data14$pn401a,
  postsecondary = 0,
  seniorsecondary = 0,
  juniorsecondary = 0,
  primary = 0,
  illiterate = 0,
  y10 = 0,
  y12 = 0,
  y14 = 1,
  y16 = 0,
  Shanghai = 0,
  Liaoning = 0,
  Guangdong = 0,
  Gansu = 0,
  Others = 0)

mincer14 <- merge(mincer14, latest_edu, all.x = TRUE)
mincer14 <- filter(mincer14, edu14 > 0)
mincer14$edu <- mincer14$edu14
mincer14 <- select(mincer14, -edu10, -edu12, -edu14, -edu16)

for (row in seq_len(nrow(mincer14))) {
  marriage <- data14[row, "qea0"]
  if (marriage == 2) {
    mincer14[row, "married"] <- 1
  }
  edu <- mincer14[row, "edu"]
  if (edu == 1) {
    mincer14[row, "illiterate"] <- 1
  } else if (edu == 2) {
    mincer14[row, "primary"] <- 1
  } else if (edu == 3) {
    mincer14[row, "juniorsecondary"] <- 1
  } else if (edu == 4) {
    mincer14[row, "seniorsecondary"] <- 1
  } else if (edu > 4) {
    mincer14[row, "postsecondary"] <- 1
  }
  prov <- mincer14[row, "prov"]
  if (prov == 21) {
    mincer14[row, "Liaoning"] <- 1
  } else if (prov == 31) {
    mincer14[row, "Shanghai"] <- 1
  } else if (prov == 44) {
    mincer14[row, "Guangdong"] <- 1
  } else if (prov == 62) {
    mincer14[row, "Gansu"] <- 1
  } else {
    mincer14[row, "Others"] <- 1
  }
}

In [42]:
mincer16 <- data.frame(
  pid = data16$pid,
  income = data16$income,
  age = data16$cfps_age,
  gender = data16$cfps_gender,
  urban = data16$urban16,
  prov = data16$ear201a,
  ethnic = data16$pa701code,
  married = 0,
  party = data16$qn4001,
  postsecondary = 0,
  seniorsecondary = 0,
  juniorsecondary = 0,
  primary = 0,
  illiterate = 0,
  y10 = 0,
  y12 = 0,
  y14 = 0,
  y16 = 1,
  Shanghai = 0,
  Liaoning = 0,
  Guangdong = 0,
  Gansu = 0,
  Others = 0)

mincer16 <- merge(mincer16, latest_edu, all.x = TRUE)
mincer16 <- filter(mincer16, edu16 > 0)
mincer16$edu <- mincer16$edu16
mincer16 <- select(mincer16, -edu10, -edu12, -edu14, -edu16)

for (row in seq_len(nrow(mincer16))) {
  marriage <- data16[row, "qea0"]
  if (marriage == 2) {
    mincer16[row, "married"] <- 1
  }
  edu <- mincer16[row, "edu"]
  if (edu == 1) {
    mincer16[row, "illiterate"] <- 1
  } else if (edu == 2) {
    mincer16[row, "primary"] <- 1
  } else if (edu == 3) {
    mincer16[row, "juniorsecondary"] <- 1
  } else if (edu == 4) {
    mincer16[row, "seniorsecondary"] <- 1
  } else if (edu > 4) {
    mincer16[row, "postsecondary"] <- 1
  }
  prov <- mincer16[row, "prov"]
  if (prov == 21) {
    mincer16[row, "Liaoning"] <- 1
  } else if (prov == 31) {
    mincer16[row, "Shanghai"] <- 1
  } else if (prov == 44) {
    mincer16[row, "Guangdong"] <- 1
  } else if (prov == 62) {
    mincer16[row, "Gansu"] <- 1
  } else {
    mincer16[row, "Others"] <- 1
  }
}

In [49]:
write_dta(mincer10, "~/Desktop/China-Education-Wages/CFPSData/mincer10.dta",
 version = 14, label = attr(data, "label"))
write_dta(mincer12, "~/Desktop/China-Education-Wages/CFPSData/mincer12.dta",
 version = 14, label = attr(data, "label"))
write_dta(mincer14, "~/Desktop/China-Education-Wages/CFPSData/mincer14.dta",
 version = 14, label = attr(data, "label"))
write_dta(mincer16, "~/Desktop/China-Education-Wages/CFPSData/mincer16.dta",
 version = 14, label = attr(data, "label"))